
## <center> Открытый курс по машинному обучению. Сессия № 3

### <center> Автор материала: Александр Ничипоренко

## <center> Индивидуальный проект по анализу данных </center>

Данные лежат здесь: https://yadi.sk/d/mJbzt5pV3Uf5Zt

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score,classification_report,f1_score,roc_auc_score,roc_curve,precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier as lgbmc
from catboost import CatBoostClassifier as catc
from xgboost import XGBClassifier as xgbc
plt.rcParams['figure.figsize'] = (20,20)
sns.set(style="darkgrid");
%matplotlib inline

###  Часть 1. Описание набора данных и признаков.

Одной из главных целей для любой компании является удержание своих клиентов. В торговле успехом данного процесса является совершение повторных покупок клиентами в интервал времени, который характеризует потребление различных видов товаров:

- Продукты: каждый день - еженедельно;
- Хозтовары: каждые 2 недели - месяц;
- Одежда: раз в три месяца - раз в полгода;
- Крупная и дорогая электроника: раз в 1-2 года;
- Автомобиль: раз в 3-5 лет.

В данном проекте будут исследованы данные одного заказчика (менеджеров крупного интернет-гипермаркета, основным ассортиментом которого являются товары повседневного спроса) и построена модель, предсказывающая вероятность оттока клиента.

Заказчик определил отток таким образом: клиент не сделает повторный заказ в течение трёх месяцев.
Такая постановка обусловлена тем, что почти 80% клиентов делают свой повторный заказ в течение 3-х месяцев. Таким образом поставлена цель научиться определять 20% клиентов, которые этого не сделают. 

После этого, уже можно разрабатывать различные подходы к стимулированию данного пула клиентов к повторной сделке с помощью различных маркетинговых методов.

Данные были получены от заказчика и сведены в один DataFrame. Посмотрим на них.



In [ ]:
df = pd.read_csv('data.csv',index_col='Client')

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
for index,value in enumerate(df.columns):
    print (index,":",value)

Как видно у нас **273** столбца, целевая переменная - **target**, 271 - количественный и 1 категориальный признак("Y M" = "Год Месяц").
Каждая строка - описание клиента (история его покупок за текущий и предыдущие 6 месяцев) в месяц последней покупки.

#### Теперь о количественных признаках.

###### Сокращения:
- **R** - Revenue - Выручка от продажи;
- **S** - Strings - Кол-во строк - разных позиций (артикулов);
- **O** - Orders - Кол-во заказов;
- **Q** - Qnt - Кол-во штук;

- **R_1 ... R_6, R_NOW**- Выручка по месяцам. NOW - месяц, соответствующий Y_M, _6 - предыдущий, _1 - 6 месяцев назад.
- **Month**: от 1 до 12 (январь-декабрь).
- **Y** или **N** в **R_Y_NOW, O_Y_NOW, R_N_NOW, R_N_NOW** - выручка/заказы в зависимости от способа оформления заказа. **Y** - через сайт, **N** - по телефону.

- **Orders-1003 ... Orders-Other** - заказы за 7 месяцев (от _1 до _NOW) по выделенной группе товаров (70 "топовых" групп: 1003, ..., 931) или по остальным (Other). Аналогично и с выручкой и кол-ву штук.

- **Other** - Кол-во групп товаров, купленных за 7 месяцев, входящих в группу "Other".

###  Часть 2. Первичный анализ признаков

Посмотрим количество пропусков в данных. Как видно их нет.

In [ ]:
sum(df.isnull().sum())

Посмотрим среднее количество "отточных клиентов" в наборе данных.

In [ ]:
print ('% клиентов, склонных к оттоку:', round(df['target'].mean()*100,2))

Получается даже меньше 20%. Выборка не сбалансирована.

##### Посмотрим сколько у нас "отточных" клиентов ежемесячно.

In [ ]:
churn=pd.crosstab(index=df['Y_M'],columns=df['target'])
churn['%']=round(churn[1]/churn[0]*100,2)

In [ ]:
churn.T

In [ ]:
churn_m=pd.crosstab(index=df['Month'],columns=df['target'])
churn_m['%']=round(churn_m[1]/churn_m[0]*100,2)

In [ ]:
churn_m.T

Видна некоторая сезонность, в конце года "отточных" клиентов больше, летом - меньше. Осенью клиенты делают закупки активнее.

###### Числовых показателей у нас много. Будем рассматривать их небольшими группами.
Для начала посмотрим на статистичекское описание ежемесячных показателей.

In [ ]:
df.drop(columns=['target','Month']).iloc[:,:29].describe().T

###### Можно заметить, что:

- Масштабы признаков сильно различаются (наиболее сильно: выручка и заказы);
- Есть отрицательные значения - это возвраты;
- Статистики по предыдущим периодам сильно скошены (среднее больше медианы) из-за того, что много нулей и величины распределены не нормально;
- Максимальные значения - очень сильно отличаются от средних, кто-то покупает покупает много товаров или дорогие товары;
- Обычно клиенты делают один заказ в течение месяца;
- В предыдущий месяц (_6) клиенты заказывают меньше, чем в другие предыдущие.

##### Посмотрим на значения показателей (выручка, заказы) по способу оформления заказа.

In [ ]:
df.drop(columns=['target','Month']).iloc[:,29:29+14*2].describe().T

Заметно, что преобладают заказы, оформленные через Интернет.

#### Посмотрим разницу по показателям в зависимости от целевого признака.

In [ ]:
ch_1=df[df['target']==1].drop(columns=['target','Month']).iloc[:,:29].describe().T

In [ ]:
ch_0=df[df['target']==0].drop(columns=['target','Month']).iloc[:,:29].describe().T

In [ ]:
ch_0-ch_1

Как видно, клиенты, которые нас интересуют - покупают меньше. В особенности, в предыдущие периоды.

##### Посмотрим, какие товары заказывают и на какие товары тратят деньги наши клиенты.

In [ ]:
goods=pd.pivot_table(data=df,values=df.iloc[:,202:273],columns=df['target'],aggfunc=np.sum)
goods['%_churn']=goods[1]/goods[0]*100
goods.sort_values(by='%_churn',ascending=False).head(10)

Как видно, по разным категориям товаров доля затраченных денег отличается. Таким образом, если клиент потратил сумму на какую-то группу товаров, то вероятность его ухода как понижается, так и повышается в зависимости от этой группы.

In [ ]:
goods_ord=pd.pivot_table(data=df,values=df.iloc[:,60:131],columns=df['target'],aggfunc=np.sum)
goods_ord['%_churn']=goods_ord[1]/goods_ord[0]*100
goods_ord.sort_values(by='%_churn',ascending=False).head(10)

In [ ]:
sns.heatmap(np.corrcoef(goods_ord['%_churn'],goods['%_churn']));

Как видно, заказы и деньги по товарным категориям коррелируют.

###### Посмотрим, сколько товарных групп из "Other" покупают разные клиенты.

In [ ]:
Other_0=df[df['target']==0]['Other'].describe()
Other_1=df[df['target']==1]['Other'].describe()
Other=pd.concat([Other_0,Other_1],axis=1,names=['Total','1'])
Other.columns=['0','1']
Other

Как видно, уходящие клиенты покупают обычно в два раза меньше товаров из категории "Другое".

###  Часть 3. Первичный визуальный анализ признаков

Визуализируем распределение целевого класса.

In [ ]:
plt.figure(figsize=[8, 5])
sns.countplot(df['target']);

Далее будем исследовать распределения признаков в зависимости от значения **"target"**. Для скошенных влево распределений будем применять **log(1+x)** преобразование и отсекать экстремально большие значения (>95%-99% квантили).

In [ ]:
plt.figure(figsize=[20, 15])
for i in range(1,8):
    plt.subplot(3, 3, i)
    sns.distplot(np.log1p(df[df['target']==1].iloc[:,i].apply(lambda x: 0 if x<0 else x)),kde=False,norm_hist=True,color='r',label='target: 1')
    sns.distplot(np.log1p(df[df['target']==0].iloc[:,i].apply(lambda x: 0 if x<0 else x)),kde=False,norm_hist=True,color='g',label='target: 0')
    plt.legend()
    plt.title('log1x')

In [ ]:
plt.figure(figsize=[20, 15])
for i in range(1,8):
    plt.subplot(3, 3, i)
    sns.distplot(df[(df['target']==1) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.95))].iloc[:,i].apply(lambda x: 0 if x<0 else x),kde=False,norm_hist=True,color='r',label='target: 1')
    sns.distplot(df[(df['target']==0) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.95))].iloc[:,i].apply(lambda x: 0 if x<0 else x),kde=False,norm_hist=True,color='g',label='target: 0')
    plt.legend()
    plt.title('<95% quantile')

In [ ]:
plt.figure(figsize=[20, 15])
for i in range(1+8,8+8):
    plt.subplot(3, 3, i-8)
    sns.distplot(np.log1p(df[df['target']==1].iloc[:,i].apply(lambda x: 0 if x<0 else x)),kde=False,norm_hist=True,color='r',label='target: 1')
    sns.distplot(np.log1p(df[df['target']==0].iloc[:,i].apply(lambda x: 0 if x<0 else x)),kde=False,norm_hist=True,color='g',label='target: 0')
    plt.legend()
    plt.title('log1x')

In [ ]:
plt.figure(figsize=[20, 15])
for i in range(1+8,8+8):
    plt.subplot(3, 3, i-8)
    sns.distplot(df[(df['target']==1) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.95))].iloc[:,i],kde=False,norm_hist=True,color='r',label='target: 1')
    sns.distplot(df[(df['target']==0) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.95))].iloc[:,i],kde=False,norm_hist=True,color='g',label='target: 0')
    plt.legend()
    plt.title('<95% quantile')

In [ ]:
plt.figure(figsize=[20, 15])
for i in range(1+8+7,8+8+7):
    plt.subplot(3, 3, i-8-7)
    sns.distplot(np.log1p(df[df['target']==1].iloc[:,i].apply(lambda x: 0 if x<0 else x)),kde=False,norm_hist=True,color='r',label='target: 1')
    sns.distplot(np.log1p(df[df['target']==0].iloc[:,i].apply(lambda x: 0 if x<0 else x)),kde=False,norm_hist=True,color='g',label='target: 0')
    plt.legend()
    plt.title('log1x')

In [ ]:
plt.figure(figsize=[20, 15])
for i in range(1+8+7,8+8+7):
    plt.subplot(3, 3, i-8-7)
    sns.distplot(df[(df['target']==1) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.99))].iloc[:,i],kde=False,norm_hist=True,color='r',label='target: 1')
    sns.distplot(df[(df['target']==0) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.99))].iloc[:,i],kde=False,norm_hist=True,color='g',label='target: 0')
    plt.legend()
    plt.title('<99% quantile')

In [ ]:
plt.figure(figsize=[20, 15])
for i in range(1+8+7+7,8+8+7+7):
    plt.subplot(3, 3, i-8-7-7)
    sns.distplot(np.log1p(df[df['target']==1].iloc[:,i].apply(lambda x: 0 if x<0 else x)),kde=False,norm_hist=True,color='r',label='target: 1')
    sns.distplot(np.log1p(df[df['target']==0].iloc[:,i].apply(lambda x: 0 if x<0 else x)),kde=False,norm_hist=True,color='g',label='target: 0')
    plt.legend()
    plt.title('log1x')

In [ ]:
plt.figure(figsize=[20, 15])
for i in range(1+8+7+7,8+8+7+7):
    plt.subplot(3, 3, i-8-7-7)
    sns.distplot(df[(df['target']==1) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.95))].iloc[:,i].apply(lambda x: 0 if x<0 else x),kde=False,norm_hist=True,color='r',label='target: 1')
    sns.distplot(df[(df['target']==0) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.95))].iloc[:,i].apply(lambda x: 0 if x<0 else x),kde=False,norm_hist=True,color='g',label='target: 0')
    plt.legend()
    plt.title('<95% quantile')

**По распределениям показателей за месяца можно увидеть, что:**
- В месяц "NOW" у "отточных" клиентов сумма отгрузки, кол-во артикулов, штук и заказов меньше;
- В месяц "NOW" у "отточных" клиентов пик на уровне 4000;
- В предыдущие месяца у "отточных клиентов" меньше заказов и они реже;

In [ ]:
plt.figure(figsize=[20, 15])
for i in range(1+8+7+7+8,8+8+7+7+8):
    plt.subplot(3, 3, i-8-7-7-8)
    sns.distplot(df[(df['target']==1) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.95))].iloc[:,i].apply(lambda x: 0 if x<0 else x),kde=False,norm_hist=True,color='r',label='target: 1')
    sns.distplot(df[(df['target']==0) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.95))].iloc[:,i].apply(lambda x: 0 if x<0 else x),kde=False,norm_hist=True,color='g',label='target: 0')
    plt.legend()
    plt.title('<95% quantile')

In [ ]:
plt.figure(figsize=[20, 15])
for i in range(1+8+7+7+8+7,8+8+7+7+8+7):
    plt.subplot(3, 3, i-8-7-7-8-7)
    sns.distplot(df[(df['target']==1) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.95))].iloc[:,i].apply(lambda x: 0 if x<0 else x),kde=False,norm_hist=True,color='r',label='target: 1')
    sns.distplot(df[(df['target']==0) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.95))].iloc[:,i].apply(lambda x: 0 if x<0 else x),kde=False,norm_hist=True,color='g',label='target: 0')
    plt.legend()
    plt.title('<95% quantile')

In [ ]:
plt.figure(figsize=[20, 15])
for i in range(1+8+7+7+8+7+7,8+8+7+7+8+7+7):
    plt.subplot(3, 3, i-8-7-7-8-7-7)
    sns.distplot(df[(df['target']==1) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.99))].iloc[:,i].apply(lambda x: 0 if x<0 else x),kde=False,norm_hist=True,color='r',label='target: 1')
    sns.distplot(df[(df['target']==0) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.99))].iloc[:,i].apply(lambda x: 0 if x<0 else x),kde=False,norm_hist=True,color='g',label='target: 0')
    plt.legend()
    plt.title('<99% quantile')

In [ ]:
plt.figure(figsize=[20, 15])
for i in range(1+8+7+7+8+7+7+7,8+8+7+7+8+7+7+7):
    plt.subplot(3, 3, i-8-7-7-8-7-7-7)
    sns.distplot(df[(df['target']==1) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.99))].iloc[:,i].apply(lambda x: 0 if x<0 else x),kde=False,norm_hist=True,color='r',label='target: 1')
    sns.distplot(df[(df['target']==0) & (df.iloc[:,i]<df.iloc[:,i].quantile(0.99))].iloc[:,i].apply(lambda x: 0 if x<0 else x),kde=False,norm_hist=True,color='g',label='target: 0')
    plt.legend()
    plt.title('<99% quantile')

По способу оформления заказа клиенты мало отличаются, все предпочитают - Интернет.

In [ ]:
sns.factorplot(x='Other',y='target',data=df,kind='bar',size=5,aspect=2.8);

По распределению доли отточных клиентов, видна обратная зависимость от количества товарных групп (ТГ) из "Другое", чем меньше ТГ - тем больше доля оттока.

In [ ]:
sns.factorplot(x='Month',y='target',data=df,kind='bar',size=4,aspect=2.2);

Доля оттока зависит от рассматриваемого месяца текущей закупки.

In [ ]:
plt.figure(figsize=[35, 35])
sns.heatmap(df.drop(columns=['target','Month']).corr(),cmap="RdBu_r");

Из корреляционной матрицы видно, что есть коррелирующие признаки:
- Заказы, выручка и штуки по товарным группам и по времени;
- Одинаковые признаки по времени;

###  Часть 4. Закономерности, "инсайты", особенности данных.

Найдены и выдвинуты предположения о природе различных корреляций/пропусков/закономерностей и выбросов, найденных в предыдущих пунктах. Есть пояснение, почему они важны для решаемой задачи;

**По проведённому анализу можно сделать выводы:**
- Клиенты, которые не закупятся в ближайшие 3 месяца, реже закупались в предыдущие полгода;
- Данные клиенты покупали меньше товаров из категории "Other";
- Пик распределения суммы отгрузки в момент "NOW" у данных клиентов находится в районе 4000;
- Есть сезонная (по месяцам) зависимость;
- Признаки, связанные со способом оформления заказа малоинформативны;
- Есть корреляции заказов, выручки и штук по товарам, скорее всего эти признаки не повышают качество;
- "Выбросы" - это "VIP" клиенты для компании, а не ошибка в данных.

###  Часть 5. Выбор метрики.

Для задач бинарной классификации обычно используются следующие метрики:

- Доля правильных ответов(Accuracy)
- Полнота(Recall)
- Точность (Precision)
- Среднее гармоническое Recall и Precision (F1-Score)
- LogLoss
- Площадь по ROC-кривой (ROC-AUC)

В данной задаче целевой класс несбалансирован (85%/15%).Также для применения модели в жизни нужно оценивать вероятность того, что клиент не сделает покупку в следующем промежутке времени, чтобы выбрать оптимальный порог принятия решения на кого воздействовать различными способами. Таким образом, наиболее подходящей метрикой является **ROC-AUC**.


###  Часть 6. Выбор модели.

В качестве моделей будут использоваться:

- Логистическую регрессию(LogisticRegression)
- Случайный лес(RandomForestClassifier)
- Градиентный бустинг над деревьями (xgboost, lightgbm)

В принципе, это наиболее часто используемые модели для задач классификаций, где обучающая выборка не очень большая и не сильно разряженная. У каждой модели есть свои плюсы для решаемой задачи:

- Логистическая регрессия хороша тем, что легко можно запустить в production, есть интепритируемость;
- Случайный лес - даёт хорошее качество без настройки гиперпараметров, не склонен к переобучению;
- Градиентный бустинг - возможность получить лучшее качество при настройки гиперпараметров. В данной задаче попробуем две реализации данного типа модели.

###  Часть 7-9. Предобработка данных. Кросс-валидация и настройка гиперпараметров модели.Создание новых признаков и описание этого процесса

Проведена предобработка данных для конкретной модели. При необходимости есть и описано масштабирование признаков, заполнение пропусков, замены строк на числа, OheHotEncoding, обработка выбросов, отбор признаков с описанием используемых для этого методов. Корректно сделано разбиение данных на обучающую и отложенную части;

**Из категориальных кризнаков у нас только месяц, сделаем OHE. Удалим столбец "Month".**

In [ ]:
df=pd.concat([df,pd.get_dummies(df['Month'], prefix='M', prefix_sep='_')],axis=1)
df=df.drop(columns='Month')

У нас данные представлены за период июль 2016 - декабрь 2017. Для корректной валидации нужно учитывать временную составляющую. На всякий случай отсортируем данные по столбцу "Y_M" и удалим этот признак.

In [ ]:
df=df.sort_values(by='Y_M')
df.head()

In [ ]:
df.tail()

In [ ]:
df=df.drop(columns='Y_M')

Разобьём всю выборку на обучающую и проверочную в соотношении 9:1. В проверочную часть попадут данные за 2 последних месяца.

In [ ]:
idx = int(df.shape[0]*0.9)
df_train = df.iloc[:idx,:]
df_valid = df.iloc[idx:,:]

In [ ]:
df.shape,df_train.shape,df_valid.shape

Как уже говорилиось, у нас есть временная зависимость, поэтому для правильной валидации будем использовать sklearn.TimeSeriesSplit. Сделаем 10 фолдов, чтобы валидироваться на выборке соизмеримой с df_valid.

In [ ]:
tscv=TimeSeriesSplit(n_splits=10)

Посмотрим на качество моделей без настроек параметров на исходных признаках, для LogisticRegression сделаем стандартизацию признаков.

In [ ]:
X_train = df_train.drop(columns='target')
y_train = df_train['target']

In [ ]:
lr = LogisticRegression()
rf = RandomForestClassifier()
lg = lgbmc()
xg = xgbc()
cb = catc()

In [ ]:
std = StandardScaler()
lr_pipeline = make_pipeline(std,lr)

In [ ]:
%%time
lrcv = cross_val_score(lr_pipeline,X_train,y_train,cv=tscv,scoring='roc_auc')
rfcv = cross_val_score(rf,X_train,y_train,cv=tscv,scoring='roc_auc')
lgcv = cross_val_score(lg,X_train,y_train,cv=tscv,scoring='roc_auc')
xgcv = cross_val_score(xg,X_train,y_train,cv=tscv,scoring='roc_auc')

In [ ]:
print ('lr_cv_score',np.mean(lrcv),"+-",np.std(lrcv))
print ('rf_cv_score',np.mean(rfcv),"+-",np.std(rfcv))
print ('lg_cv_score',np.mean(lgcv),"+-",np.std(lgcv))
print ('xg_cv_score',np.mean(xgcv),"+-",np.std(xgcv))

На кроссвалидации лучший результат оказался у xgboost, худший - у случайного леса, логистическая регрессия показала результат немного ниже, чем у бустинга.

Попробуем улучшить результат с помощью добавлением новых признаков и убиранием малоинформативных, а также настройкой гиперпараметров.

Создадим признаки: Общее количество заказов за период, количество закупок (в месяцах), количество "топовых" товарных групп за 7 месяцев.

In [ ]:
df_train['Orders_total']=df_train.iloc[:,15:22].sum(axis=1)

In [ ]:
sns.distplot(np.log1p(df_train[df_train['target']==1]['Orders_total'].apply(lambda x: 0 if x<0 else x)),kde=False,norm_hist=True,color='r');
sns.distplot(np.log1p(df_train[df_train['target']==0]['Orders_total'].apply(lambda x: 0 if x<0 else x)),kde=False,norm_hist=True,color='g');

In [ ]:
df_train['B_M']=np.array([df_train.iloc[:,i].apply(lambda x: 1 if x >0 else 0) for i in range(0,6)]).sum(axis=0)

In [ ]:
sns.factorplot(y='target',x='B_M',data=df_train,kind='bar');

In [ ]:
df_train['TG_total']=np.array([df_train.iloc[:,i].apply(lambda x: 1 if x >0 else 0) for i in range(200,270)]).sum(axis=0)

In [ ]:
sns.factorplot(y='target',x='TG_total',data=df_train,kind='bar',aspect=5);

По всем трём признакам видна закономерность - чем меньше значение, тем выше доля оттока.
Проверим качество с добавлением этих признаков для xgboost.

In [ ]:
X_train = df_train.drop(columns='target')
y_train = df_train['target']

In [ ]:
xgcv = cross_val_score(xg,X_train,y_train,cv=tscv,scoring='roc_auc')

In [ ]:
print ('xg_cv_score',np.mean(xgcv),"+-",np.std(xgcv))

Качество не улучшилось. Настроим параметры на кросс-валидации.

In [ ]:
XG_params = {'n_estimators': [100,200,300,400,500],
            'seed':[17],
            'max_depth': [3,4,5,6,7,8],
            'learning_rate': [0.01,0.05,0.1]}

In [ ]:
xggs = GridSearchCV(xg,param_grid=XG_params,scoring='roc_auc',cv=tscv)

###  Часть 10. Прогноз для тестовой или отложенной выборки

In [ ]:
xg.fit(X_train,y_train)
df_valid['Orders_total']=df_valid.iloc[:,15:22].sum(axis=1)
df_valid['B_M']=np.array([df_valid.iloc[:,i].apply(lambda x: 1 if x >0 else 0) for i in range(0,6)]).sum(axis=0)
df_valid['TG_total']=np.array([df_valid.iloc[:,i].apply(lambda x: 1 if x >0 else 0) for i in range(200,270)]).sum(axis=0)

X_valid = df_valid.drop(columns='target')
y_valid = df_valid['target']

y_pred_proba=xg.predict_proba(X_valid)[:,1]

In [ ]:
roc_auc_score(y_valid,y_pred_proba)

Результат получился выше, чем на валидации. Это связано с тем, что у нас была TimeSeriesValidation.

### Часть 11. Выводы 

После исследования и построения модели можно сделать следюущие выводы:
    
1)В принципе, получена модель с неплохим качеством, которая может помогать определять клиентов, которые не закупятся в ближайшие 3 месяца.

2)Возможные пути улучшения модели - добавить признаки другого типа (взаимодействия с клиентами), покрутить признаки.

3)Логистическая регрессия показала неплохие результаты, можно попробовать немного поднастроить её вместо построения многих деревьев.
